# Independent vs. Non-independent performance testing

In [2]:
import torch
from torch.distributions import Distribution
import torch.nn as nn
from torch.distributions import Normal, Laplace, Bernoulli, Gamma,Cauchy
from causalflows.flows import CausalFlow
from typing import Callable
from zuko.flows import UnconditionalDistribution
from sklearn.model_selection import KFold
import copy
from torch.utils.data import DataLoader, TensorDataset
from architectures import get_stock_transforms
from csuite import SCMS, SCM_DIMS, SCM_MASKS
from causal_cocycle.causalflow_helper import select_and_train_flow, sample_do, sample_cf
from causal_cocycle.helper_functions import ks_statistic, wasserstein1_repeat, rmse

In [4]:
scm = SCMS['triangle_linear']
dims = SCM_DIMS['triangle_linear']
# ----------------------------------------
# 1) Define sample size and SCM dimension
# ----------------------------------------
N = 1000  # number of samples
# For a 5‐chain, d = 5 implicitly in scm

# --------------------------------------------------
# 2) Build noise distributions and transforms
# --------------------------------------------------

# (a) U₁ ∼ N(0,1)
dist_u1 = Normal(0,1)

# (b) “Base” for (U₂,…,U₅) is also N(0,1), but we will sample shape (N,4)
dist_base = Cauchy(0,1)

# (c) tf_u1: identity map on raw1 (shape (N,) → (N,1))
tf_u1 = lambda raw1: raw1.view(-1, 1)

# (d) tf_rest: affine map on raw_rest ∈ ℝ⁴ to induce correlation among U₂…U₅
rho = 0.0
Σ = (1-rho) * torch.eye(2) + rho * torch.ones(2, 2)
L = torch.linalg.cholesky(Σ)  # L is lower‐triangular, shape (4,4)

# We want U_rest = raw_rest @ Lᵀ
tf_rest = lambda raw_rest: raw_rest @ L.t()

# Package them into lists of length 2
noise_dists      = [dist_u1, dist_base]
noise_transforms = [tf_u1,   tf_rest]

# --------------------------------------------------
# 3) Draw data from the 5‐chain linear SCM
# --------------------------------------------------
#    - No intervention (intervention_node=None)
#    - return_u=True so we also get back the U matrix of shape (N,5)

X_full, U_full = scm(
    N=N,
    seed=42,
    intervention_node=None,
    intervention_value=None,
    return_u=True,
    noise_dists=noise_dists,
    noise_transforms=noise_transforms
)

# X has shape (N, 5), U_full has shape (N, 5)
# U_full[:,0] = U₁, U_full[:,1] = U₂, ..., U_full[:,4] = U₅

print("X.shape =", X_full.shape)
print("U_full.shape =", U_full.shape)

# --------------------------------------------------
# 4) Quick sanity check: empirical correlation among U₂ and U₃
# --------------------------------------------------
U2 = U_full[:, 1]
U3 = U_full[:, 2]
emp_corr_23 = torch.corrcoef(torch.stack([U2, U3]))[0,1].item()
print(f"Empirical corr(U₂, U₃) ≈ {emp_corr_23:.3f}  (target ρ = {rho})")

X.shape = torch.Size([1000, 3])
U_full.shape = torch.Size([1000, 3])
Empirical corr(U₂, U₃) ≈ -0.001  (target ρ = 0.0)


# Training flow model

In [3]:
num_epochs = 1000
k_folds = 2
batch_size = 64
lr = 1e-2
transforms = get_stock_transforms(x_dim=0, y_dim=dims)  # returns a list of 4 different MAF instances
Base = Normal
base = UnconditionalDistribution(
    Base, loc=torch.zeros(dims), scale=torch.ones(dims), buffer=True
)
flows = [CausalFlow(transform=maf, base=base)
         for maf in transforms
        ]
# CV + retrain across all transforms
best_flow, test_nll, best_idx, cv_scores = select_and_train_flow(
    flows, X_full, train_fraction=1.0, k_folds=k_folds,
    num_epochs=num_epochs, batch_size=batch_size, lr=lr,
    device=X_full.device
)

In [4]:

# ───────────────────────────────────────────────────────────────
# 1. Re-generate observational data (X_full = [X1, Y1, ..., Y4])
# ───────────────────────────────────────────────────────────────
N = 10000
X_obs_full, _ = scm(
    N=N,
    seed=0,
    intervention_node=None,
    return_u=True,
    noise_dists=noise_dists,
    noise_transforms=noise_transforms
)

X_obs = X_obs_full[:, :1]     # (N,1)
Y_obs = X_obs_full[:, 1:]     # (N,4)

# ───────────────────────────────────────────────────────────────
# 2. Generate ground-truth interventional data under do(X1 = c)
# ───────────────────────────────────────────────────────────────
c = 0.0
X_int_full = scm(
    N=N,
    seed=0,
    intervention_node=1,
    intervention_value=c,
    return_u=False,
    noise_dists=noise_dists,
    noise_transforms=noise_transforms
)
Y_true_int = X_int_full[:, 1:]  # (N,4)

# ───────────────────────────────────────────────────────────────
# 3. Generate model interventional samples using sample_do
# ───────────────────────────────────────────────────────────────
device = X_obs.device
flow_model = best_flow.to(device)

with torch.no_grad():
    X_model_int = sample_do(
        flow=best_flow,
        index=0,
        intervention_fn=lambda old: torch.full_like(old, c),
        sample_shape=torch.Size([N])
    )
Y_model_int = X_model_int[:, 1:]

# ───────────────────────────────────────────────────────────────
# 4. Compute marginal KS statistics
# ───────────────────────────────────────────────────────────────
ks_vals = []
for j in range(4):
    ks_j = ks_statistic(Y_model_int[:, j].cpu(), Y_true_int[:, j].cpu())
    ks_vals.append(ks_j)
print("Interventional KS per dimension:", ks_vals)

# ───────────────────────────────────────────────────────────────
# 5. Generate counterfactual predictions
# ───────────────────────────────────────────────────────────────
Z_obs = X_obs_full.to(device)
with torch.no_grad():
    Z_cf = sample_cf(
        flow=best_flow,
        x_obs=Z_obs,
        index=0,
        intervention_fn=lambda old: torch.full_like(old, c)
    )
Y_cf_pred = Z_cf[:, 1:]

# Get true counterfactuals for these same samples
Z_cf_true = scm(
    N=N,
    seed=0,  # same seed as earlier for consistency
    intervention_node=1,
    intervention_value=c,
    return_u=False,
    noise_dists=noise_dists,
    noise_transforms=noise_transforms
).to(device)
Y_cf_true = Z_cf_true[:, 1:]

# ───────────────────────────────────────────────────────────────
# 6. Compute RMSE per dimension
# ───────────────────────────────────────────────────────────────
rmse_vals = []
for j in range(4):
    rmse_j = torch.sqrt(((Y_cf_pred[:, j] - Y_cf_true[:, j]) ** 2).nanmean()).item()
    rmse_vals.append(rmse_j)
print("Counterfactual RMSE per dimension:", rmse_vals)


Interventional KS per dimension: [0.018599987030029297, 0.1550999879837036, 0.11640000343322754, 0.0974000096321106]
Counterfactual RMSE per dimension: [0.007982463575899601, 0.003497605910524726, 0.051112715154886246, 0.1640557199716568]


## Training Cocycle Models

In [6]:
# Split into context X_ctx (shape (N,1)) and Y (shape (N,4)):
X_ctx = X_full[:, 0:1]   # 1‐dimensional context
Y_obs = X_full[:, 1:]    # 4‐dimensional output

# 2) Import the stock autoregressive transforms for a 1→4 conditional flow

# Each MaskedAutoregressiveTransform expects:
#   • features = y_dim = 4
#   • context  = x_dim = 1
transforms = get_stock_transforms(x_dim=1, y_dim=2)  # returns a list of 4 different MAF instances

# 3) Wrap each AR transform into a 1‐layer ZukoCocycleModel
from causal_cocycle.model_new import ZukoCocycleModel

# If you want to cross‐validate over all 4 architectures, build a list:
cocycle_models = [
    ZukoCocycleModel(nn.ModuleList(maf))  
    for maf in transforms
]
# Alternatively, to pick just the first architecture:
# cocycle_model = ZukoCocycleModel(nn.ModuleList([transforms[0]]))
model0 = cocycle_models[0]

from torch.utils.data import random_split

# 4) Build a CMMD-V loss via CocycleLossFactory
from causal_cocycle.loss_factory import CocycleLossFactory
from causal_cocycle.kernels import gaussian_kernel

# We supply two Gaussian kernels (one on X, one on U):
kernel = [gaussian_kernel(), gaussian_kernel()]
loss_factory = CocycleLossFactory(kernel)

# We need to “tune” the kernels’ lengthscales on training data.
# Split X_ctx / Y_obs into train / val
X_train = X_ctx
Y_train = Y_obs

# Build the CMMD-V loss, which will internally set lengthscales via median heuristic on (X_train,Y_train)
cmmdv_loss = loss_factory.build_loss("CMMD_V", X_train, Y_train, subsamples=10000)

# 5) Call optimise(...)
from causal_cocycle.optimise_new import optimise

# Example hyperparameters—feel free to adjust as desired:
learn_rate       = 1e-3
epochs           = 1000
weight_decay     = 0.0
batch_size       = 64
val_batch_size   = 256
scheduler        = False
schedule_milestone = 10
lr_mult          = 0.90
print_           = True
plot             = False
likelihood_param_opt = False
likelihood_param_lr  = 0.01

# Finally, run training. We supply both train and val sets so optimise can compute validation loss each epoch:
best_model, history = optimise(
    model            = model0,
    loss_tr          = cmmdv_loss,
    inputs           = X_train,
    outputs          = Y_train,
    inputs_val       = None,
    outputs_val      = None,
    learn_rate       = learn_rate,
    epochs           = epochs,
    weight_decay     = weight_decay,
    batch_size       = batch_size,
    val_batch_size   = val_batch_size,
    scheduler        = scheduler,
    schedule_milestone = schedule_milestone,
    lr_mult          = lr_mult,
    print_           = print_,
    plot             = plot,
    likelihood_param_opt = likelihood_param_opt,
    likelihood_param_lr  = likelihood_param_lr,
    loss_val         = None  # no separate “loss_val” function needed, since CMMD-V is symmetric
)

Epoch 1/1000, Training Loss: -0.4279
Epoch 2/1000, Training Loss: -0.4415
Epoch 3/1000, Training Loss: -0.4518
Epoch 4/1000, Training Loss: -0.4567
Epoch 5/1000, Training Loss: -0.4354
Epoch 6/1000, Training Loss: -0.4538
Epoch 7/1000, Training Loss: -0.4538
Epoch 8/1000, Training Loss: -0.4399
Epoch 9/1000, Training Loss: -0.4521
Epoch 10/1000, Training Loss: -0.4371
Epoch 11/1000, Training Loss: -0.4267
Epoch 12/1000, Training Loss: -0.4567
Epoch 13/1000, Training Loss: -0.4581
Epoch 14/1000, Training Loss: -0.4592
Epoch 15/1000, Training Loss: -0.4598
Epoch 16/1000, Training Loss: -0.4637
Epoch 17/1000, Training Loss: -0.4590
Epoch 18/1000, Training Loss: -0.4507
Epoch 19/1000, Training Loss: -0.4295
Epoch 20/1000, Training Loss: -0.4483
Epoch 21/1000, Training Loss: -0.4467
Epoch 22/1000, Training Loss: -0.4626
Epoch 23/1000, Training Loss: -0.4544
Epoch 24/1000, Training Loss: -0.4728
Epoch 25/1000, Training Loss: -0.4800
Epoch 26/1000, Training Loss: -0.4699
Epoch 27/1000, Traini

In [22]:
import torch
from torch.distributions import Normal

# ──────────────────────────────────────────────────────────────────────────────
# 1) Re‐generate “observational” data (no intervention)
# ──────────────────────────────────────────────────────────────────────────────
N = 10000

# draw N observations (no do‐intervention)
X_obs_full,U_obs = scm(
    N=N,
    seed=0,
    intervention_node=None,
    return_u=True,            # we do not need U here
    noise_dists=noise_dists,
    noise_transforms=noise_transforms
)
X_obs = X_obs_full[:, :1]   # (N,1)
Y_obs = X_obs_full[:, 1:]   # (N,4)


# ──────────────────────────────────────────────────────────────────────────────
# 2) Draw “true interventional” data by calling the SCM with intervention_node=1
# ──────────────────────────────────────────────────────────────────────────────
c = 1.0
X_int_full, U_int = scm(
    N=N,
    seed=0,                     # fresh seed (independent of the observational draw)
    intervention_node=1,        # hard‐set X₁ = c
    intervention_value=c,
    return_u=True,
    noise_dists=noise_dists,
    noise_transforms=noise_transforms
)
Y_true_int = X_int_full[:, 1:]  # shape (N,4)


# ──────────────────────────────────────────────────────────────────────────────
# 3) Use the trained cocycle model to predict counterfactuals under do(X₁ = c)
# ──────────────────────────────────────────────────────────────────────────────
#    Y_pred_int[i] = f( x_do = c,  f^{-1}( x_obs[i],  Y_obs[i] ) )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model = model0.to(device)       # your trained ZukoCocycleModel
x_obs      = X_obs.to(device)        # (N,1)
y_obs      = Y_obs.to(device)        # (N,4)
x_do       = torch.full_like(x_obs, c)  # (N,1), all entries = c

with torch.no_grad():
    Y_pred_int = best_model.cocycle(x_do, x_obs, y_obs)  # (N,4)

Y_pred_int_cpu = Y_pred_int.cpu()
Y_true_int_cpu = Y_true_int.cpu()


# ──────────────────────────────────────────────────────────────────────────────
# 4) Compute marginal KS‐distance for each output dimension j = 0..3
# ──────────────────────────────────────────────────────────────────────────────

ks_vals = []
for j in range(Y_true_int_cpu.size(1)):  # 4 dims
    ks_j = ks_statistic(Y_pred_int_cpu[:, j], Y_true_int_cpu[:, j])
    ks_vals.append(ks_j)

print("KS distances (hard intervention) per dimension:", ks_vals)


# ──────────────────────────────────────────────────────────────────────────────
# 5) Compute counterfactual RMSE (per output dimension)
# ──────────────────────────────────────────────────────────────────────────────
rmse_vals = []
for j in range(Y_true_int_cpu.size(1)):
    diff   = Y_pred_int_cpu[:, j] - Y_true_int_cpu[:, j]
    rmse_j = torch.sqrt((diff ** 2).mean()).item()
    rmse_vals.append(rmse_j)

print("Counterfactual RMSE (hard intervention) per dimension:", rmse_vals)

KS distances (hard intervention) per dimension: [0.0048999786376953125, 0.0027000010013580322]
Counterfactual RMSE (hard intervention) per dimension: [0.09286681562662125, 0.055022113025188446]


In [13]:
# 1) Grab the first MAF in your flow:
maf = model0.transforms[0]   # this is a MaskedAutoregressiveTransform

# 2) The very first layer inside that MAF’s conditioner is actually a MaskedLinear.
#    In most Zuko versions, it lives under `maf.hyper[0]`, so:
masked_linear = maf.hyper[0]     # type: zuko.flows.autoregressive.MaskedLinear

# 3) The raw weight parameter is:
raw_W = masked_linear.weight     # shape = (4, 5)
#    and the associated mask is:
mask  = masked_linear.mask       # same shape = (4, 5)

print("raw_W (un‐masked parameter):")
print(raw_W)

print("\nmask (0/1 binary tensor):")
print(mask)

# 4) If you multiply them, you get the actual weight used during forward:
effective_W = raw_W * mask
print("\neffective_W = raw_W * mask (i.e. the masked‐out values):")
print(effective_W)


raw_W (un‐masked parameter):
Parameter containing:
tensor([[  0.1843,   0.1667, -10.0927],
        [ -0.5592,  -0.1339,  -0.4109]], requires_grad=True)

mask (0/1 binary tensor):
tensor([[False, False,  True],
        [ True, False,  True]])

effective_W = raw_W * mask (i.e. the masked‐out values):
tensor([[  0.0000,   0.0000, -10.0927],
        [ -0.5592,  -0.0000,  -0.4109]], grad_fn=<MulBackward0>)


In [9]:
# Multi-cocycle Training

In [19]:
import torch
import torch.nn as nn
from torch.utils.data import random_split, TensorDataset
from causal_cocycle.model_new import ZukoCocycleModel
from causal_cocycle.loss_factory import CocycleLossFactory
from causal_cocycle.kernels import gaussian_kernel
from causal_cocycle.optimise_new import optimise
from architectures import get_stock_transforms

# -----------------------------
# Step 1: Setup from full data
# -----------------------------
# Full data matrix (N, 5), e.g. from chain5
N, d = X_full.shape
assert d == 3
X_full = X_full.to("cuda" if torch.cuda.is_available() else "cpu")

# We will train 4 cocycle models: each V_j | V_{<j}, for j=1..4
models = []
losses = []
factory = CocycleLossFactory([gaussian_kernel(), gaussian_kernel()])
subsamples = 1000

# ------------------------------------------
# Step 2: Construct model/loss per coordinate
# ------------------------------------------
for j in range(1, d):
    x_ctx = X_full[:, :j]         # V_{<j}
    y_out = X_full[:, j:j+1]      # V_j

    # split train/val
    dataset = TensorDataset(x_ctx, y_out)
    n_train = int(1 * N)
    n_val = N - n_train
    train_ds, val_ds = random_split(dataset, [n_train, n_val], generator=torch.Generator().manual_seed(0))
    X_train = torch.stack([train_ds[i][0] for i in range(n_train)])
    Y_train = torch.stack([train_ds[i][1] for i in range(n_train)])

    # Make transform and model
    maf = get_stock_transforms(x_dim=j, y_dim=1)[0]
    model = ZukoCocycleModel(nn.ModuleList(maf)).to(X_full.device)
    models.append(model)

    # Tune kernel and get CMMD-V loss
    loss_j = factory.build_loss("CMMD_V", X_train, Y_train, subsamples=subsamples)
    losses.append(loss_j)

# ------------------------------------------
# Step 3: Define aggregate loss over models
# ------------------------------------------
class CMMDMultiLoss(nn.Module):
    def __init__(self, models, losses):
        super().__init__()
        self.models = models
        self.losses = losses

    def forward(self, _, inputs, outputs):
        loss_total = 0.0
        for j, (model, loss) in enumerate(zip(self.models, self.losses)):
            x_j = inputs[:, :j+1]       # context: V_{<j+1}
            y_j = outputs[:, j:j+1]     # target:  V_{j+1}
            loss_total += loss(model, x_j, y_j)
        return loss_total

# ------------------------------------------
# Step 4: Training
# ------------------------------------------
multi_loss = CMMDMultiLoss(models, losses)
composite_model = nn.ModuleList(models)

best_model, history = optimise(
    model              = composite_model,
    loss_tr            = multi_loss,
    inputs             = X_full[:, :d-1],  # V_{<d}
    outputs            = X_full[:, 1:],    # V_1 to V_4
    inputs_val         = None,
    outputs_val        = None,
    learn_rate         = 1e-2,
    epochs             = 500,
    weight_decay       = 0.0,
    batch_size         = 64,
    val_batch_size     = 256,
    scheduler          = False,
    schedule_milestone = 10,
    lr_mult            = 0.90,
    print_             = True,
    plot               = False,
    likelihood_param_opt = False,
    likelihood_param_lr  = 0.01,
    loss_val           = None
)

Epoch 1/500, Training Loss: -0.7753
Epoch 2/500, Training Loss: -0.9125
Epoch 3/500, Training Loss: -1.0557
Epoch 4/500, Training Loss: -1.0881
Epoch 5/500, Training Loss: -1.0913
Epoch 6/500, Training Loss: -1.1070
Epoch 7/500, Training Loss: -1.0996
Epoch 8/500, Training Loss: -1.0906
Epoch 9/500, Training Loss: -1.1183
Epoch 10/500, Training Loss: -1.1287
Epoch 11/500, Training Loss: -1.1140
Epoch 12/500, Training Loss: -1.1163
Epoch 13/500, Training Loss: -1.1405
Epoch 14/500, Training Loss: -1.1653
Epoch 15/500, Training Loss: -1.1440
Epoch 16/500, Training Loss: -1.1458
Epoch 17/500, Training Loss: -1.1488
Epoch 18/500, Training Loss: -1.1504
Epoch 19/500, Training Loss: -1.1502
Epoch 20/500, Training Loss: -1.1766
Epoch 21/500, Training Loss: -1.1587
Epoch 22/500, Training Loss: -1.1906
Epoch 23/500, Training Loss: -1.1932
Epoch 24/500, Training Loss: -1.2042
Epoch 25/500, Training Loss: -1.1907
Epoch 26/500, Training Loss: -1.2438
Epoch 27/500, Training Loss: -1.2100
Epoch 28/5

In [23]:
import torch
from torch.distributions import Normal

# 1) Re-generate observational and interventional data
N = 5000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# observational data (X₁,...,X₅)
X_obs_full, U = scm(
    N=N, seed=0, intervention_node=None, return_u=True,
    noise_dists=noise_dists, noise_transforms=noise_transforms
)
X_obs_full = X_obs_full.to(device)

# interventional (true) data under do(X₁ = c)
c = 1.0
X_true_cf_full, U_cf = scm(
    N=N, seed=0, intervention_node=1, intervention_value=c,
    return_u=True, noise_dists=noise_dists, noise_transforms=noise_transforms
)
Y_true_cf = X_true_cf_full[:, 1:]  # (N, 4)

# 2) Predict counterfactuals from models
models = [f.to(device) for f in models]  # trained cocycle models per coordinate

# Intervention: set X₁ = c
X_obs_cf = X_obs_full.clone()
X_obs_cf[:, 0] = c   # hard intervention on V₀

# Sequentially reconstruct (V₁,...,V₄) using the trained cocycles
predicted_cf = torch.zeros((N, 2), device=device)

for j, model in enumerate(models):
    # Build context using previously predicted counterfactuals
    if j == 0:
        x_new = X_obs_cf[:, :1]                     # just the intervened x₁
        x_old = X_obs_full[:, :1]
    else:
        x_new = torch.cat([X_obs_cf[:, :1], predicted_cf[:, :j]], dim=1)  # updated context
        x_old = torch.cat([X_obs_full[:, :1], X_obs_full[:, 1:1+j]], dim=1)

    y_old = X_obs_full[:, j+1:j+2]  # observed y_j

    with torch.no_grad():
        y_pred = model.cocycle(x_new, x_old, y_old)

    predicted_cf[:, j:j+1] = y_pred  # store for use in future iterations

# 3) Compute KS distances
def ks_statistic(a: torch.Tensor, b: torch.Tensor) -> float:
    a, b = a.flatten(), b.flatten()
    a_s, _ = torch.sort(a)
    b_s, _ = torch.sort(b)
    all_vals = torch.cat([a_s, b_s]).unique()
    cdf_a = torch.bucketize(all_vals, a_s, right=True).float() / a_s.numel()
    cdf_b = torch.bucketize(all_vals, b_s, right=True).float() / b_s.numel()
    return (torch.abs(cdf_a - cdf_b).max()).item()

ks_vals = [
    ks_statistic(predicted_cf[:, j].cpu(), Y_true_cf[:, j].cpu())
    for j in range(2)
]
print("KS distances (hard intervention) per dimension:", ks_vals)

# 4) Compute RMSE
rmse_vals = [
    torch.sqrt(((predicted_cf[:, j] - Y_true_cf[:, j])**2).mean()).item()
    for j in range(2)
]
print("Counterfactual RMSE (hard intervention) per dimension:", rmse_vals)

KS distances (hard intervention) per dimension: [0.005800008773803711, 0.008400022983551025]
Counterfactual RMSE (hard intervention) per dimension: [0.07469481229782104, 0.17490257322788239]


In [21]:
for i in range(4):
    print(models[i].transforms[0].hyper[0].weight)
    print(torch.linalg.solve(X_full[:,:i+1].T @ X_full[:,:i+1], X_full[:,:i+1].T @ X_full[:,i+1:i+2]))

Parameter containing:
tensor([[-10.0753]], requires_grad=True)
tensor([[7.6951]])
Parameter containing:
tensor([[-1.0113, -0.5126]], requires_grad=True)
tensor([[4.2480],
        [0.4998]])


IndexError: list index out of range

In [1]:
import torch
import torch.nn as nn
import torch
from torch.distributions import (
    Distribution, Normal, Laplace, Cauchy, Gamma, Uniform
)
# Csuite Imports
from csuite_mixed import SCMS, SCM_DIMS, SCM_MASKS
from architectures import get_stock_transforms
from causal_cocycle.loss_factory import CocycleLossFactory
from causal_cocycle.optimise_new import validate
from causal_cocycle.kernels import gaussian_kernel
from causal_cocycle.model_new import ZukoCocycleModel
from causal_cocycle.helper_functions import ks_statistic, wasserstein1_repeat, rmse


def evaluate_models(
    models_dict: dict,
    index_dict: dict,
    sc_fun: callable,
    X: torch.Tensor,
    Y: torch.Tensor,
    seed: int = None,
    N_true: int = 10**5,
    intervention_node: int = 1,
    intervention_value: float = 0.0,
) -> dict:
    """
    Evaluates each CocycleModel on interventional, counterfactual, and paired-difference metrics.
    Uses ground-truth SCM (`sc_fun`) for reference.
    """
    if seed is not None:
        torch.manual_seed(seed)
    device = X.device
    N, d = X.shape
    _, p = Y.shape

    # --- True paired (Y, Y_cf) via SCM generator ---
    torch.manual_seed(seed)
    X_obs_true, U = sc_fun(
        N_true,
        seed=seed,
        intervention_node=None,
        return_u=True,
    )
    X_obs_true = X_obs_true.to(device)
    torch.manual_seed(seed)
    X_cf_true, Ucf = sc_fun(
        N_true,
        seed=seed,
        intervention_node=intervention_node,
        intervention_value=intervention_value,
        return_u=True,
    )
    X_cf_true = X_cf_true.to(device)
    # check same noise
    assert((U-Ucf).sum()==0)
    
    # Extract paired Y variables (columns 1 onward)
    Y_true = X_obs_true[:, 1:]
    Y_cf_true = X_cf_true[:, 1:]
    Y_dim = Y_true.shape[1]
 
    # paired-difference
    diff_true = Y_cf_true - Y_true
    
    results = {}
    with torch.no_grad():
        for name, (model, _) in models_dict.items():
    
            # Counterfactuals with cocycle(X', X, Y)
            Y_model = model.cocycle(X, X, Y)
            Y_cf_model = model.cocycle(X*0 + intervention_value, X, Y)
            diff_model = Y_cf_model - Y_model  # shape (N, p)
            
            # --- Marginal KS values for each Y dimension ---
            w1_vals = [
                wasserstein1_repeat(diff_model[:, j].cpu(), diff_true[:, j].cpu())
                for j in range(Y_dim)
            ]
    
            # --- Marginal KS values for each Y dimension ---
            ks_vals = [
                ks_statistic(diff_model[:, j].cpu(), diff_true[:, j].cpu())
                for j in range(Y_dim)
            ]
    
            # --- Interventional marginal KS ---
            w1_int_vals = [
                wasserstein1_repeat(Y_cf_model[:, j].cpu(), Y_cf_true[:, j].cpu())
                for j in range(Y_dim)
            ]
            ks_int_vals = [
                ks_statistic(Y_cf_model[:, j].cpu(), Y_cf_true[:, j].cpu())
                for j in range(Y_dim)
            ]
    
            # ---- Counterfactual RMSE using cocycle model ----
            torch.manual_seed(seed)
            X_obs, U = sc_fun(
                N,
                seed=seed,
                intervention_node=None,
                return_u=True,
            )
            X_obs = X_obs.to(device)
            Y_obs = X_obs[:, 1:].to(device)
            
            torch.manual_seed(seed)
            X_cf, Ucf = sc_fun(
                N,
                seed=seed,
                intervention_node=intervention_node,
                intervention_value=intervention_value,
                return_u=True,
            )
            X_cf = X_cf.to(device)
            Y_cf_true = X_cf[:, 1:]
            
            # Consistency check
            Z = torch.cat([X, Y], dim=1).to(device)
            assert((Z - X_obs).sum() == 0)
                
            rmse_cf_vals = [
                rmse(Y_cf_model[:, j].cpu(), Y_cf_true[:, j].cpu())
                for j in range(Y_dim)
            ]
    
            results[name] = {
                'KS_CF': ks_vals,
                'KS_int': ks_int_vals,
                'W1_CF': w1_vals,
                'W1_int': w1_int_vals,
                'RMSE_CF': rmse_cf_vals,
                'index': index_dict[name][0]
            }
    
    return results

def run_experiment(
    sc_name: str = "chain5_linear",
    corr = 0.0,
    seed: int = 0,
    N: int = 1000,
    use_dag = False,
    num_epochs: int = 1000,
    k_folds: int = 2,
    batch_size: int = 128,
    lr: float = 1e-2,
) -> dict:
    """
    Runs experiments across all SCMs for a given noise distribution, seed, and sample size N.

    Returns a dict mapping sc_name to its metrics.
    """
    torch.manual_seed(seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Cocycle opt configs
    opt_config = {
    "epochs": num_epochs,            # Number of epochs per optimisation run.
    "batch_size": batch_size,         # Training batch size.
    "val_batch_size": 1024,         # Training batch size.
    "learn_rate": lr,       # Base learning rate for model parameters.
    "print_": False          # AMEND THIS
}

    # Getting SCM
    sc_fun = SCMS[sc_name]
    d = SCM_DIMS[sc_name]
    mask = SCM_MASKS[sc_name][1:][:,1:] if (use_dag and d > 2) else None
    
    # generate data
    V, U = sc_fun(
        N, seed=seed, return_u=True,
    )
    V = V.to(device)
    X, Y = V[:,:1], V[:,1:]

    # Build architectures
    transforms = get_stock_transforms(x_dim = 1, y_dim = d-1, mask = mask)

    # Cocycle models
    cocycle_models = [ZukoCocycleModel(nn.ModuleList(t)) for t in transforms]

    # Set up loss factories
    kernel = [gaussian_kernel()] * 2
    loss_factory = CocycleLossFactory(kernel)

    # Cross-validate each family
    # Cocycle CMMD (flow-specific LR's_
    hyper_kwargs = [{'learn_rate': lr}] + [{'learn_rate': lr * 1}] * (len(cocycle_models)-1)
    cmmdv = loss_factory.build_loss("CMMD_V", X, Y)
    final_v, (idx_v, _) = validate(cocycle_models, cmmdv, X, Y, method="CV",
                                   train_val_split=0.5, opt_kwargs=opt_config,
                                   hyper_kwargs=hyper_kwargs, choose_best_model="overall", retrain=True)

    # Collect models & indices
    models = {
        'Cocycle_CMMD_V': (final_v, 'cmmdv'),
    }
    idxs = {
        'Cocycle_CMMD_V': (idx_v, 'cmmdv'),
    }

    metrics = evaluate_models(models, idxs, sc_fun, X, Y, seed=seed)
    metrics.update({'scm': sc_name})

    return metrics

In [2]:
results = run_experiment(sc_name = "fork_nonlinear", 
                         seed = 0,
                         N = 2000)

Model 0, Fold 0: Validation Loss = -0.4575
Model 0, Fold 1: Validation Loss = -0.4601
Model 1, Fold 0: Validation Loss = -0.4705
Model 1, Fold 1: Validation Loss = -0.4726
Model 2, Fold 0: Validation Loss = -0.4731
Model 2, Fold 1: Validation Loss = -0.4721
Model 3, Fold 0: Validation Loss = -0.4692
Model 3, Fold 1: Validation Loss = nan
Best overall model index: 3 with average validation loss nan
Retraining best model on full dataset...


KeyboardInterrupt: 

In [32]:
results

{'Cocycle_CMMD_V': {'KS_CF': [0.01203000545501709, 0.011069953441619873],
  'KS_int': [0.016780003905296326, 0.018119990825653076],
  'W1_CF': [tensor(0.1902), tensor(0.1069)],
  'W1_int': [tensor(3.4722), tensor(8.8068)],
  'RMSE_CF': [0.061543382704257965, 0.06613373756408691],
  'index': 0},
 'noise': 'cauchy',
 'scm': 'triangle_linear'}